In [1]:
! pip install pandas

1.	**¿Qué porcentaje de trabajos requieren experiencia en 'Remote' o 'Hybrid' en comparación con los trabajos presenciales?**  
Enfoque: Análisis de proporciones. Se calculará la proporción de trabajos remotos, híbridos y para obtener porcentajes exactos. (estaba en el ejemplo, pero fue literalmente el enfoque)


In [3]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

total_jobs = len(df)
remote_or_hybrid_jobs = df[df['remote_modality'].isin(['fully_remote', 'hybrid'])].shape[0]
percentage_remote_or_hybrid = (remote_or_hybrid_jobs / total_jobs) * 100
print(f"\nPorcentaje de trabajos 'Remote' o 'Hybrid': {percentage_remote_or_hybrid:.2f}%")



Porcentaje de trabajos 'Remote' o 'Hybrid': 65.57%


2. **¿Cuáles son las tres habilidades más comúnmente solicitadas en la categoría de 'Data Science / Analytics'?**  
Vamos a revisar la columna de skills (solo texto de descripción) y vamos a filtrar caracteres no deseados. Adicional a eso, haremos una lista de palabras clave que queremos excluir del conjunto más común. Una vez hecho esto, vamos a obtener la lista de trabajos y eliminaremos todos los que tengan el conjunto skills vacío. Luego, haremos una lista con todas las palabras y, después de filtrar las palabras comunes, contaremos cuántas veces aparece cada palabra y seleccionaremos las tres más comunes.


In [2]:
import pandas as pd
import re
from collections import Counter

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Función para limpiar y extraer palabras de la columna skills
def extract_words(skills):
        words = re.sub(r'<[^>]+>', '', skills)  # Eliminar etiquetas HTML
        words = re.sub(r'[^a-zA-Z0-9\s]', '', words)  # Eliminar caracteres no alfanuméricos
        return words.lower().split()

    # Lista de palabras comunes a excluir
stop_words = set(['li', 'p', 'de', 'en', 'y', 'br', 'con' ,'ul','experiencia','o','la','strong','para','el','a','como'])

    # Filtrar trabajos de 'Data Science / Analytics'
data_science_jobs = df[df['category'] == 'Data Science / Analytics'].dropna(subset=['skills'])

    # Extraer y contar palabras
all_words = data_science_jobs['skills'].apply(extract_words).explode()
filtered_words = all_words[~all_words.isin(stop_words)]
top_three_words = Counter(filtered_words).most_common(3)

print("\nTres palabras más comúnmente mencionadas en las descripciones de 'Data Science / Analytics':")
for word, count in top_three_words:
    print(f"{word}: {count}")



Tres palabras más comúnmente mencionadas en las descripciones de 'Data Science / Analytics':
datos: 103
sql: 47
data: 38


3. **¿Cuáles trabajos requieren certificaciones específicas (como PL-300 para Power BI)?**  
Enfoque: Vamos a crear un set de keywords con palabras que representan certificaciones y revisaremos si estan en el campo skills (que es solo el texto del elemento description)
iteramos por todos los trabajos y si existen subiremos el contador +1, además se extrae los company_name y skills y se muestran


In [8]:
import pandas as pd
import re

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Lista de palabras clave para certificaciones específicas
certification_keywords = ['pl-300', 'aws certified', 'pmp', 'cissp', 'scrum master', 'cfa', 'six sigma', 'aws']

# Convertir todas las entradas de la columna 'skills' a minúsculas
df['skills'] = df['skills'].str.lower()

# Filtrar trabajos que requieren certificaciones específicas
df['requires_certification'] = df['skills'].apply(lambda x: any(keyword in x for keyword in certification_keywords) if pd.notna(x) else False)
certification_filtered = df[df['requires_certification']]

# Contar trabajos que requieren certificaciones específicas
certification_experience_count = certification_filtered.shape[0]

print(f"\nNúmero de trabajos que requieren certificaciones específicas: {certification_experience_count}")
print("\nTrabajos que requieren certificaciones específicas:")
certification_filtered[['id', 'company_name', 'skills']]



Número de trabajos que requieren certificaciones específicas: 149

Trabajos que requieren certificaciones específicas:


,id,company_name,skills
3,desarrollador-power-bi-data-lake-y-bd-lisit-sa...,Lisit,"div, para, el, cargo, a, postular, contamos, c..."
12,data-governance-specialist-coderslab-io-buenos...,coderslab.io,"p, conocimientos, br, cloud, aws, azure, o, gc..."
14,arquitecto-de-datos-senior-hibrido-bogota-soph...,Sophos Solutions,"p, requerimos, p, ul, li, profesional, en, en,..."
20,ingeniero-a-bi-y-proyectos-copec-s-a-santiago,Copec S.A,"ul, li, titulo, profesional, en, las, de, civi..."
21,data-analyst-mas-analytics-santiago-b0ea,MAS Analytics,"div, al, ser, en, necesitamos, que, tengas, ha..."
...,...,...,...
917,lead-machine-learning-neuralworks-santiago,NeuralWorks,"ul, li, al, menos, 4, de, experiencia, como, m..."
918,generative-ai-consultancy-lead-softserve-remote,Softserve,"h2, strong, if, you, are, br, strong, br, h2, ..."
923,machine-learning-engineer-haystack-news-lima,Haystack News,"ul, li, bachelor, s, degree, in, computer, sci..."
924,machine-learning-engineer-interfell-remote,Interfell,"ul, li, 3, de, experiencia, en, desarrollo, de..."


4. **¿Qué porcentaje de trabajos requiere un nivel avanzado de inglés?**  

Enfoque: Identificar las palabras clave comunes en las descripciones de habilidades que indican un nivel avanzado de inglés, luego contar los trabajos que mencionan estas palabras clave, y finalmente calcular la proporción de estos trabajos en relación con el total de trabajos disponibles.


In [4]:
import pandas as pd

df = pd.read_csv('../data/all_jobs_data.csv')

advanced_english_keywords = ['advanced, english', 'nivel, avanzado, de, inglés', 'english, proficiency', 'fluency, in, english', 'advanced, level, english','ingles, avanzado']

if 'skills' in df.columns:
    df.loc[:, 'skills'] = df['skills'].str.lower()
    
    advanced_english_count = df['skills'].apply(lambda x: any(keyword in x for keyword in advanced_english_keywords) if pd.notna(x) else False).sum()
    
    percentage_advanced_english = (advanced_english_count / len(df)) * 100
    
    print(f"\nPorcentaje de trabajos que requieren un nivel avanzado de inglés: {percentage_advanced_english:.2f}%")
else:
    print("\nNo se encontró la columna 'skills' en el dataset.")


Porcentaje de trabajos que requieren un nivel avanzado de inglés: 1.81%


5. **¿Cuántos trabajos requieren experiencia previa específica en la industria (por ejemplo, agrícola, ciberseguridad)?**  
Enfoque: Vamos a crear un set de keywords con palabras que representan experiencia en la industria y revisaremos si estan en el campo skills (que es solo el texto del elemento description)
iteramos por todos los trabajos y si existen subiremos el contador +1

In [5]:
import pandas as pd
import re

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

industry_keywords = ['agrícola', 'ciberseguridad', 'finanzas', 'salud', 'educación', 'tecnología']
#a minusculas
df['skills'] = df['skills'].str.lower()
    
#Obtener la cantidad de trabajos que requieren experiencia previa
industry_experience_count = df['skills'].apply(lambda x: any(keyword in x for keyword in industry_keywords) if pd.notna(x) else False).sum()
    
print(f"\nNúmero de trabajos que requieren experiencia previa específica en la industria: {industry_experience_count}")




Número de trabajos que requieren experiencia previa específica en la industria: 37


6. **¿Cuáles trabajos en 'Design / UX' requieren experiencia en herramientas específicas como Figma o Adobe Illustrator?**  
Enfoque: armamos un set de keywords, con nombres de herramientas especificas, con los cuales vamos a iterar por la columna skills que tiene la del trabajo, de html convertida a texto.
luego vamos a iterar por todos los trabajos filtrados de design ux y revisaremos si contiene alguna de las keywords

In [5]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Palabras clave de herramientas de diseño
design_tools_keywords = ['figma', 'adobe illustrator', 'sketch', 'photoshop']

# Filtrar trabajos de 'Design / UX'
design_jobs = df[df['category'] == 'Design / UX']

# Convertir todas las entradas de la columna 'skills' a minúsculas
design_jobs['skills'] = design_jobs['skills'].str.lower()

# Filtrar trabajos que requieren experiencia en herramientas específicas
specific_tools_experience = design_jobs[design_jobs['skills'].apply(lambda x: any(keyword in x for keyword in design_tools_keywords) if pd.notna(x) else False)]

# Contador de trabajos totales y específicos
total_design_jobs = len(design_jobs)
specific_tools_count = len(specific_tools_experience)

print(f"\nNúmero total de trabajos en 'Design / UX': {total_design_jobs}")
print(f"Número de trabajos que requieren experiencia en herramientas específicas como Figma o Adobe Illustrator: {specific_tools_count}")

print("\nTrabajos en 'Design / UX' que requieren experiencia en herramientas específicas como Figma o Adobe Illustrator:")
print(specific_tools_experience[['id', 'skills']])



Número total de trabajos en 'Design / UX': 36
Número de trabajos que requieren experiencia en herramientas específicas como Figma o Adobe Illustrator: 26

Trabajos en 'Design / UX' que requieren experiencia en herramientas específicas como Figma o Adobe Illustrator:
                                                    id  \
639                 disenador-a-grafico-a-lfi-santiago   
640                      disenador-ux-ui-magnet-remote   
641               product-designer-encuadrado-santiago   
643       ui-and-graphic-designer-neuralworks-santiago   
644                     ui-ux-designer-ensitech-remote   
645  disenador-design-system-match-agencia-consulto...   
649                   disenador-ui-ux-citolab-santiago   
650  director-de-arte-freelance-match-agencia-consu...   
652                product-designer-alegra-remote-79e5   
653                    ux-ui-designer-sr-cmpc-santiago   
654                  product-designer-lemontech-remote   
655                 ux-ui-designer-m

C:\Users\Kivye\AppData\Local\Temp\ipykernel_16688\1020322434.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  design_jobs['skills'] = design_jobs['skills'].str.lower()


7. **¿Qué habilidades o tecnologías son más solicitadas en trabajos remotos en comparación con los trabajos presenciales?**  
Enfoque: analizar y comparar las keywords mas solicitadas en trabajos remotos vs los trabajos presenciales, definimos la lista de keywords que serian tecnologias diferentes, en este caso figma, adobe, sketch (faltarian mas, pero sirven para el ejemplo) despues revisamos hacemos dos listas, una para las que son presenciales y otra para las que son remotas, luego hacemos una cuenta de cuantos trabajos incluyen las palabras claves y tambien hacemos un conteo de las palabras mas usadas, filtramos los conectores y etiquetas html para obtener lo que serian las experiencias requeridas. finalmente mostramos el numero total de trabajos remotos y presenciales vs el numero de trabajos en cada categoria que requieren tecnologias especificas y mostramos las cinco habilidades mas mencionadas en las descripciones de los trabajos


In [10]:
import pandas as pd
import re
from collections import Counter

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Palabras clave de herramientas específicas
tech_keywords = ['figma', 'adobe illustrator', 'sketch', 'photoshop']

# Convertir todas las entradas de la columna 'skills' a minúsculas
df['skills'] = df['skills'].str.lower()

# Función para limpiar y extraer palabras de la columna skills
def extract_words(skills):
    words = re.sub(r'<[^>]+>', '', skills)  # Eliminar etiquetas HTML
    words = re.sub(r'[^a-zA-Z0-9\s]', '', words)  # Eliminar caracteres no alfanuméricos
    return words.lower().split()

# Lista de palabras comunes a excluir
stop_words = set(['li', 'p', 'de', 'en', 'y', 'br', 'con', 'ul', 'experiencia', 'o', 'la', 'strong', 'para', 'el', 'a', 'como','div','and'])

# Filtrar trabajos remotos y presenciales
remote_jobs = df[df['remote_modality'].isin(['fully_remote', 'hybrid'])]
on_site_jobs = df[~df['remote_modality'].isin(['fully_remote', 'hybrid'])]

# Contar trabajos que requieren tecnologías específicas
remote_tech_experience = remote_jobs[remote_jobs['skills'].apply(lambda x: any(keyword in x for keyword in tech_keywords) if pd.notna(x) else False)]
on_site_tech_experience = on_site_jobs[on_site_jobs['skills'].apply(lambda x: any(keyword in x for keyword in tech_keywords) if pd.notna(x) else False)]

# Contador de trabajos totales y específicos
total_remote_jobs = len(remote_jobs)
remote_tech_count = len(remote_tech_experience)

total_on_site_jobs = len(on_site_jobs)
on_site_tech_count = len(on_site_tech_experience)

print(f"\nNúmero total de trabajos remotos: {total_remote_jobs}")
print(f"Número de trabajos remotos que requieren tecnologías específicas: {remote_tech_count}")

print(f"\nNúmero total de trabajos presenciales: {total_on_site_jobs}")
print(f"Número de trabajos presenciales que requieren tecnologías específicas: {on_site_tech_count}")

# Extraer y contar palabras para trabajos remotos
remote_skills_words = remote_jobs['skills'].dropna().apply(extract_words).explode()
remote_filtered_words = remote_skills_words[~remote_skills_words.isin(stop_words)]
remote_words_counter = Counter(remote_filtered_words)
top_remote_words = remote_words_counter.most_common(5)

# Extraer y contar palabras para trabajos presenciales
on_site_skills_words = on_site_jobs['skills'].dropna().apply(extract_words).explode()
on_site_filtered_words = on_site_skills_words[~on_site_skills_words.isin(stop_words)]
on_site_words_counter = Counter(on_site_filtered_words)
top_on_site_words = on_site_words_counter.most_common(5)

print("\nHabilidades más solicitadas en trabajos remotos:")
for word, count in top_remote_words:
    print(f"{word}: {count}")

print("\nHabilidades más solicitadas en trabajos presenciales:")
for word, count in top_on_site_words:
    print(f"{word}: {count}")



Número total de trabajos remotos: 617
Número de trabajos remotos que requieren tecnologías específicas: 24

Número total de trabajos presenciales: 324
Número de trabajos presenciales que requieren tecnologías específicas: 15

Habilidades más solicitadas en trabajos remotos:
conocimiento: 397
desarrollo: 326
conocimientos: 323
datos: 257
experience: 240

Habilidades más solicitadas en trabajos presenciales:
desarrollo: 174
datos: 158
in: 151
conocimiento: 144
que: 141


8. **¿Cuántos trabajos ofrecen beneficios relacionados con la salud (como seguro de salud o dental)?**  
Enfoque: hacemos una lista de keywords y obtenemos la lista de perks que tiene cada trabajo, vamos a iterar por estas listas y compararemos con la lista de keywords para ver si existen, en caso de existir alguna de las keywords, sumaremos uno al contador, representando que incluye el perk.


In [6]:
import pandas as pd
import re

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Convertir todas las entradas de la columna 'perks' a minúsculas
df['perks'] = df['perks'].str.lower()

# Definir palabras clave relacionadas con beneficios de salud
health_keywords = ['health', 'dental', 'seguro de salud', 'seguro dental', 'cobertura médica', 'beneficios médicos']

# Contar trabajos que ofrecen beneficios relacionados con la salud
health_benefits_count = df['perks'].apply(lambda x: any(keyword in x for keyword in health_keywords) if pd.notna(x) else False).sum()

print(f"\nNúmero de trabajos que ofrecen beneficios relacionados con la salud: {health_benefits_count}")



Número de trabajos que ofrecen beneficios relacionados con la salud: 365


9. **¿Cuántos trabajos se publicaron en el último mes?**  
Enfoque: vamos a obtener el datetime de la columna published_at, luego vamos a compararlo con la fecha de 30 dias atras, si el published_at es mas nuevo que los 30 dias, se subira el contador, representando que fue publicada hace menos de un mes.

In [7]:
import pandas as pd
from datetime import datetime, timedelta

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Convertir la columna 'published_at' a fechas
df['published_at'] = pd.to_datetime(df['published_at'], unit='s')

# Calcular la fecha de hace un mes
one_month_ago = datetime.now() - timedelta(days=30)

# Filtrar trabajos publicados en el último mes
recent_jobs = df[df['published_at'] >= one_month_ago]

print(f"\nNúmero de trabajos publicados en el último mes: {len(recent_jobs)}")



Número de trabajos publicados en el último mes: 441


10. **¿Cuáles trabajos tienen la mayor cantidad de aplicaciones hasta la fecha?**  
Enfoque: vamos a ordenar los trabajos en orden descendente, por el valor de la cantidad de aplicacionesl que ya incluye el json del trabajo


In [1]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('../data/all_jobs_data.csv')

# Ordenar los trabajos por la cantidad de aplicaciones y obtener los top 5
top_jobs = df[['id', 'applications_count']].sort_values(by='applications_count', ascending=False).head(5)

print("\nTrabajos con la mayor cantidad de aplicaciones hasta la fecha:")
top_jobs



Trabajos con la mayor cantidad de aplicaciones hasta la fecha:


,id,applications_count
53,analista-de-datos-forma-moovmedia-group-remote,3782
360,desarrollador-fronted-zeroq-remote,2500
486,desarrollador-de-software-gatblac-remote,1392
357,back-end-software-engineer-jr-foris-ai-remote-...,1333
711,mailing-marketing-forma-moovmedia-group-remote,1175
